<a href="https://colab.research.google.com/github/cseeg/DiSCoVeR-SuperCon-NOMAD-SMACT/blob/main/SVC_Trained_Model_Bayesian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **SVC using Hyperparameter Tuning**

 `imread` is used for featurization of images. After standardization, a trained, bayesian tuned, SVM model is used to predict if an image is a wind turbine or not. `svc_bo` must be downloaded and imported. A folder containing any given number of images in a local directory can be used.



Imports

In [ ]:
%pip install scikit-image
%pip install sklearn

import os
from skimage.io import imread
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import plot_confusion_matrix
import pickle
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=6b15d9e26b3d5e34d83cb8c84c290d789a83532f8262bd96a89d8e670a2fb482
  Stored in directory: /root/.cache/pip/wheels/42/56/cc/4a8bf86613aafd5b7f1b310477667c1fca5c51c3ae4124a003
Successfully built sklearn


Preprocessing

In [ ]:
#mount drive if necessary
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
#Featurizes images in a folder to a Pandas DataFrame
#Assuming all pictures are in one folder called 'WT', must rename final file in directory to a list like below

Test= ['WT'] 
flat_data_arr=[]
target_arr=[]

#change directory to what is needed, if local or on Drive
datadir='/content/drive/MyDrive/USHE-S-22/WT-NWT/test'
# datadir= os.path.join(os.path.dirname(notebook_path), "/Users/jordanmiller/Documents/UVU/USHE Project/WT_vs_NWT_Phase_I/train")

In [ ]:
for i in Test:
  print(f'loading... category : {i}')
  path=os.path.join(datadir,i)
  for img in os.listdir(path):
    img_array=imread(os.path.join(path,img), as_gray=False)
#   img_resized=resize(img_array,(135,135))
    flat_data_arr.append(img_array.flatten())
    target_arr.append(Test.index(i))
  print(f'loaded category:{i} successfully')
flat_data=np.array(flat_data_arr)
target=np.array(target_arr)
df=pd.DataFrame(flat_data)
df['Target']=target
df

loading... category : WT
loaded category:WT successfully


,0,1,2,3,4,5,6,7,8,9,...,34416,34417,34418,34419,34420,34421,34422,34423,34424,Target
0,195,223,247,182,226,253,179,228,255,191,...,16,28,14,17,29,17,24,36,22,0
1,73,79,91,87,94,102,240,245,248,252,...,192,193,197,190,191,196,190,191,196,0
2,254,255,253,255,255,255,255,255,255,254,...,191,192,196,191,192,196,191,192,196,0
3,255,255,255,254,253,255,255,254,255,255,...,186,186,188,186,186,186,187,187,187,0
4,112,150,51,85,119,45,51,79,38,60,...,216,217,222,218,217,222,218,217,222,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
775,230,255,255,231,250,255,239,250,254,234,...,203,204,209,203,204,209,203,204,209,0
776,236,250,253,236,250,253,236,250,253,239,...,196,196,198,201,201,203,199,199,201,0
777,255,255,255,255,255,255,255,255,255,255,...,208,204,203,214,206,195,213,204,195,0
778,49,71,25,33,53,18,20,38,16,19,...,100,129,75,101,130,76,100,129,75,0


In [ ]:
x_test=df.iloc[:,:-1]
y_test=df.iloc[:,-1]
print('Splitted Successfully')

Splitted Successfully


Standardization

In [ ]:
#Initiate scaler
sc = StandardScaler()

#Standardize the testing dataset
x_test_t = pd.DataFrame(sc.fit_transform(x_test),index=x_test.index, columns=x_test.columns)

Import model and Predict

In [ ]:
#change path to where model is downloaded

model=pickle.load(open('/content/drive/MyDrive/USHEF22/svc_bo','rb'))

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator SVC from version 0.23.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,


In [ ]:
#if not provided with labeled data
prediction = model.predict(x_test_t)

In [ ]:
confidence = model.predict_proba(x_test_t)

In [ ]:
#write to text file:
f = open("output.txt", "w")
f.write("Below is a report for each image in the given input file.")
f.write("\n")
f.write("For each image, a confidence rating is given.")
f.write("\n")
f.write("A prediction of 0 is True Negative and 1 is True Positive.")
f.write("\n")
f.write("\n")
im=0
j=-1
for i in confidence:
  im = im+1
  j= j+1
  f.write(f"Image {im}")
  f.write("\n")
  f.write(f"Confidence: {i}")
  f.write("\n")
  f.write(f"Prediction: {prediction[j]}")
  f.write("\n")
  f.write("\n")
f.close()